In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#extract data
# !unzip /content/drive/My\ Drive/modified-mnist.zip -d /content/drive/My\ Drive/modifiedmnist

In [0]:
#Import required packages
import numpy as np
import pandas as pd
import os,sys
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import copy
import time
from torch.autograd import Variable
from collections import OrderedDict
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from torch.functional.F import 

In [0]:
torch.cuda.empty_cache()

In [4]:
#need to install/update imgaug package
!pip install -U git+https://github.com/aleju/imgaug.git

  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-6wuk2wrd
  Running command git clone -q https://github.com/aleju/imgaug.git /tmp/pip-req-build-6wuk2wrd
     |████████████████████████████████| 22.1MB 417kB/s 
  Created wheel for imgaug: filename=imgaug-0.3.0-cp36-none-any.whl size=876731 sha256=cbeb1dba8b51dd6ad3eb8fdc34ecfdfed20e8c28ec9a23b347cff98c7370d17c
  Stored in directory: /tmp/pip-ephem-wheel-cache-79jcj6_n/wheels/65/3d/94/ee32cbeaa29c473a4db74c2d21904ac747311fdca4732665f0
Successfully built imgaug
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.3.0 which is incompatible.
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
from imgaug import augmenters as iaa
import imgaug as ia
 

Loading and preprocessing Data

In [0]:
train_img=pd.read_pickle('/content/drive/My Drive/modifiedmnist/train_max_x')
test_img=pd.read_pickle('/content/drive/My Drive/modifiedmnist/test_max_x')


In [0]:
train_labels=pd.read_csv('/content/drive/My Drive/modifiedmnist/train_max_y.csv')
del train_labels['Id']

In [0]:
train_labels=np.array(train_labels)

In [0]:
train_labels=train_labels.reshape(-1)

In [0]:
X_train,X_val,y_train,y_val=train_test_split(train_img,train_labels,test_size=0.2,random_state=7)

In [11]:
X_train= X_train.astype('uint8')
X_val= X_val.astype('uint8')
X_train.dtype

dtype('uint8')

In [0]:
test_img=test_img.astype('uint8')

Creating a dataset class

In [0]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None,transform1=None):
        self.data = data
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        self.transform1=transform1
    def __getitem__(self, index):
        x = self.data[index]
        
        y = self.target[index]
        
        if self.transform:
            
            x = self.transform(x)
            x=x.repeat(3,1,1)
        if self.transform1:
            x=self.transform1(x)
#             print(x.shape)
            # x.unsqueeze_(0)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

Image Augmentation class of transforms

In [0]:
class ImgAugTransform:
  def __init__(self):
    # we can change the amount of augmentation by adjusting the probability
    self.aug = iaa.Sequential([iaa.Sometimes(0.5,iaa.Affine(
            scale={"x": (0.8, 1.1), "y": (0.8, 1.1)}, 
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, 
            rotate=(-10, 10), 
            shear=(-10, 10),
            order=[0, 1], 
            cval=(0, 255),
            mode='wrap' 
        )),iaa.Sometimes(0.6,iaa.OneOf([
                    iaa.GaussianBlur((0, 2.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=False),
                    iaa.Dropout((0.01, 0.1), per_channel=False),
                    iaa.Add((-10, 10), per_channel=False), # blur image using local means with kernel sizes between 2 and 7
                    iaa.edges.Canny(alpha=0.5,hysteresis_thresholds=(60, 140), sobel_kernel_size=3,colorizer=iaa.RandomColorsBinaryImageColorizer(color_true=255,color_false=0)), # blur image using local medians with kernel sizes between 2 and 7
                ])),
        ],random_order=True)
      
  def __call__(self, img):
#     img = np.array(img)
    return self.aug.augment_image(img)
    # iaa.Sometimes(0.2,iaa.Affine(rotate=(-20, 20), mode='edge'))
    # iaa.GaussianBlur(sigma=0.5),

In [0]:
transforms1=torchvision.transforms.Compose([
    ImgAugTransform(),                                        
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(224, interpolation=2),
    torchvision.transforms.ToTensor(),
    
])


In [0]:
transforms2=torchvision.transforms.Compose([
    #ImgAugTransform(),
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(224, interpolation=2),
    torchvision.transforms.ToTensor(),
    
])


In [0]:
transforms3=transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

Defining Dataloaders

In [0]:
traindataset = MyDataset(X_train, y_train,transform=transforms1,transform1=transforms3)
trainloader = DataLoader(
    traindataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

In [0]:
valdataset = MyDataset(X_val, y_val,transform=transforms2,transform1=transforms3)
valloader = DataLoader(
    valdataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

In [0]:
dset_sizes={}
dset_sizes['train']=X_train.shape[0]
dset_sizes['val']=X_val.shape[0]

The model class

In [0]:
class MyResNet(nn.Module):

    def __init__(self, in_channels=1):
        super(MyResNet, self).__init__()

        # bring resnet
        self.model = torchvision.models.resnet50(pretrained=True)
        # for param in self.model.parameters():
        #     param.requires_grad = False
        # self.model.classifier[6] = nn.Linear(4096,10)
        n_inputs = self.model.fc.in_features
        classifier = nn.Sequential(
                      nn.Linear(n_inputs, 512), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(512, 10))
      
        self.model.fc = classifier
        
    def forward(self, x):
        return self.model(x)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
model=MyResNet()
model=model.to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:00<00:00, 203MB/s]


IN CASE YOU NEED TO TEST EFFICIENTNET.

In [27]:
# ! pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /usr/local/lib/python3.6/dist-packages (0.5.1)


In [35]:
# def efficientnet(version,num_classes):
#     model = EfficientNet.from_pretrained('efficientnet-b{}'.format(version), num_classes=num_classes)
#     num_ftrs = model._fc.in_features
#     model._fc = nn.Linear(num_ftrs, num_classes)
#     return model.cuda()
# from efficientnet_pytorch import EfficientNet
# model = efficientnet(5,10)

Loaded pretrained weights for efficientnet-b5


Defining hyperparameters

In [0]:
criterion = nn.CrossEntropyLoss()
n_epochs=40
learning_rate=0.01
# optimizer=optim.Adam(model.parameters(),lr=learning_rate)

optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1,patience=2,verbose=True)
use_gpu=torch.cuda.is_available()

In [0]:
def save_checkpoint(state, filename):
    print ("=> Saving a new best")
    torch.save(state, filename) 

In [0]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model = model
    best_acc = 0.0
    best_loss=1.0
    # run for given number of epochs
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            total=0
            correct=0
            if phase == 'train':
                phase_ = trainloader
            else:
                phase_ = valloader

            # Run through all data in mini batches
            for idx,data in enumerate(phase_):
                # get the inputs
                inputs, labels = data
#                 print(labels)
                # wrap them in Variable
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), \
                        Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)

                # calculating the loss
#                 print(outputs,labels)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics for printing
                #running_loss += loss.data[0]
                running_loss += loss.item()
                total += labels.size(0)
                correct += (preds == labels).sum().item()
                running_corrects += torch.sum(preds == labels.data)
                if idx%100==0 and idx>0:
                  print("accuracy={}".format(100*correct/total))

            epoch_loss = running_loss / dset_sizes[phase]
            # if phase == 'val':
            #   val_loss=epoch_loss
            epoch_acc = running_corrects / dset_sizes[phase]
            new_acc=100*correct/total
            print('{} Loss: {:.4f} {} Acc: {:.4f}'.format(phase,epoch_loss,phase,new_acc))

            # save model if it performed better than
            # any other previous model
            if phase == 'val' and epoch_loss < best_loss:
                best_acc = epoch_acc
                best_loss=epoch_loss
                best_model = copy.deepcopy(model)
                filename='/content/drive/My Drive/res50_'+str(epoch)+'.pth.tar'
                save_checkpoint({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'best_accuracy': best_acc,
                'best_loss':best_loss,
                'optimizer_state_dict': optimizer.state_dict()
                },filename)
            elif phase=='val' and epoch_loss> best_loss:
                print('model did not improve from {:.4f} val loss'.format(best_loss))
        scheduler.step(epoch_loss)    
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model

RUN MODEL TRAINING

In [27]:
model_conv = train_model(model, criterion, optimizer, num_epochs=n_epochs)


Epoch 0/39
----------
accuracy=37.46905940594059
accuracy=51.694651741293534
accuracy=58.959717607973424
accuracy=63.81701995012469
accuracy=67.35903193612775
accuracy=69.9355241264559
accuracy=72.03994293865905
accuracy=73.57990012484395
accuracy=74.8196448390677
accuracy=76.12075424575424
accuracy=77.16564486830154
accuracy=77.98449208992507
train Loss: 0.0220 train Acc: 78.3075
accuracy=92.20297029702971
accuracy=92.63059701492537
accuracy=92.88828903654485
val Loss: 0.0073 val Acc: 92.8300
=> Saving a new best

Epoch 1/39
----------
accuracy=89.63490099009901
accuracy=88.8837064676617
accuracy=89.30647840531562
accuracy=89.26122194513715
accuracy=89.42739520958084
accuracy=89.70985856905158
accuracy=89.75570613409415
accuracy=89.84472534332085
accuracy=90.12208657047725
accuracy=90.40646853146853
accuracy=90.5057901907357
accuracy=90.65101998334721
train Loss: 0.0103 train Acc: 90.7400
accuracy=95.42079207920793
accuracy=95.4912935323383
accuracy=95.45265780730897
val Loss: 0.0050 

KeyboardInterrupt: ignored

INFERENCE

In [0]:
# ! nvidia-smi
model=MyResNet()

In [29]:
# model = efficientnet(5,10)
model=model.to(device)
#CHANGE THE FILE NAME TO POINT TO THE CHECKPOINT TO BE LOADED
model.load_state_dict(torch.load("/content/drive/My Drive/res50_18.pth.tar")["state_dict"])
model.eval()

MyResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

TEST DATALOADER

In [0]:
class MytestDataset(Dataset):
    def __init__(self, data, transform=None,transform1=None):
        self.data = data
#         self.target = torch.from_numpy(target).long()
        self.transform = transform
        self.transform1=transform1
    def __getitem__(self, index):
        x = self.data[index]
#         y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            x=x.repeat(3,1,1)
        if self.transform1:
            x=self.transform1(x)
        
        return x
    
    def __len__(self):
        return len(self.data)
testdataset = MytestDataset(test_img,transform=transforms2,transform1=transforms3)
testloader = DataLoader(
    testdataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)


CREATING CSV FOR SUBMISSION

In [31]:
ids=[]
labels=[]
for batch_idx, data in enumerate(testloader):
    ids.append(batch_idx)
    data=data.to(device)
    outputs=model(data)
    _, preds = torch.max(outputs.data, 1)
    labels.append(preds.item())
print(batch_idx)
out_df=pd.DataFrame({'Id':ids,
                    'Label':labels})
order=['Id','Label']
out_df[order].to_csv('/content/drive/My Drive/modifiedmnist/test.csv',index=False)

9999
